# Hashing
In this lecture we will go over the idea of a data structure called **hash tables** or it is also called **hashing**. When creating a hash table it is an array of some fixed size, containing the items. Each item has a **key** and addtional data fields that are associated with that key. Each **key** is mapped to an index in the array and placed in the appropriate position. For example an employee name being the key and its value being an object containing information about that employee. The mapping is called a **hash function**, which ideally should be simple to compute and should ensure that any two distinct keys get different cells. The goal of creating a hash function is such that it distributes the keys evenly among the cells in the hash table. 

<img src="./files/Hashing/hashtable.png" width="90"/>

In this example *john* hashes to 3, *phil* hashes to 4, *dave* hashes to 6, and *mary* hashes to 7. This is the basic idea of hashing. The only remaining problems are coming up with a hash function and deciding what to do when two keys map to the same value (also known as **collision**)
The advantages of hash tables are that inserting, deleting, and searching are done in constant time, O(1). This is because with the use of the hash function we know exactly where each element in the hash table is. Lets take a look at an example where hashing would come in handy. Lets say we want to count the number of occurences each character has in a string. Without hashing this would take $O(26*N)$ since first we would have to loop through each character in the alphabet and then inside that for loop, go through each character in the string and count up the frequency. 
With hashing this can be done in O(N) time. Because we can use the character as the key and the value will be it's frequency. So while looping through the hash table we simply increment the value each time. We can see the code for this below:
```python
# Create an empty array of 26 characters
hash_table = [0]*26

def hash_function(x):
    return x - 'a'

for i in range(len(string)):
    key = hash_function(string[i])
    hash_table[key] += 1
```

## Hash Function
Are simple idea for a hash function is merely returning *key mod tableSize* if the keys are integers. But sometimes the keys can have some undesirable properties. For example if the table size is 10 and all the keys divisible by 10 (i.e. they all end in a zero), then this function would be a bad choice as they would all map to 0. In this case the choice of the hash function needs to be carefully considered. This is also why it is generally a good idea to choose a prime number for the table size. <br>
Usually the keys are strings; in this case, the hash function needs to be chosen carefully. One option is to use the sum of the ASCII values of the characters in the string mod table size. This function is rather simple and quick however it is not a very good hash function. If all the keys have the same characters then they will all map to the same value creating a collision for every entry. An alternate idea is to again use them sum of ascii values but also each value is multiplied by it's position in the string then take the value mod tableSize. The algorithm for this is seen below:
```python
def hash_function(string, table_size):
    hash_val = 0
    for i in range(len(string)):
        hash_val = ascii_value(string[i]) * i

    return hash_val % table_size
```
The main detail left is collision resolution, If, when an element is inserted, it hashes to the same value as an already inserted element, then we have a **collision** and need to resolve it. We will describe two methods to resolve this: separate chaining and open addressing.

## Separate Chaining
The first strategy is known as **separate chaining**, is to keep a list of all elements that hash to the same value. For this each index would contain a doubly-linked list of keys that map to that index. The running example we will use throughout is that all the keys are perfect squares and the hashing function is key mod 10.
Below is an example with keys, 0, 81, 1, 64, 4, 25, 36, 16, 49, 9. 
<img src="./files/Hashing/separate-chaining.png" width="250"/>
To perform a search we would first get the hash value of the key then search throughout the list for that specific key. For inserting we would also have to check the list to see if the element already exist. The convention is to insert the new element at the front of the list since recent items added are most likely to be acccessed. <br>
   
## Open Addressing
Seperate chaining has the disadvantage of using linked lists. This could slow down the algorithm of searching, since searching a linked list is linear time, O(N). An alternative is to store all the values in the hash table instead of list. When a new entry has to be inserted, the hash index of the hash value is computed and we check to see if the index is empty. If it is empty we simply add the value into the index, otherwise otherwise it will **probe** the table until an empty index is found. <br>
The probe sequence is the sequence that is followed while traversing through the entries. As we will see in different probing strategies you can have different intervals between probes in the table. So when searching the table for a key the same probing sequence is followed until the key is found or an empty index is found. If we find an empty index that means the table does not contain the key. The term **open addressing** refers to the fact that the location or index of the item is not determined soley by the hash value. <br>
Since all the data goes inside the hash table, unlike separate chaining where it's stored in a list, a bigger table is needed. We call such tables that use open addressing schemes **probing hash tables**. We will now look at three common schemes. 

### Linear Probing
In **linear probing** the interval at which cells are probed is a fixed amount (usually 1). So if a collision appears we will probe the next index specified by the interval amount. Let us look at how we would calculate the index for an entry if the index is already occupied. The probe sequence is as follows: <br><br>
<center>
index = h(x) % tableSize <br>
index = (h(x) + 0) % tableSize <br>
index = (h(x) + 1) % tableSize <br>
index = (h(x) + 2) % tableSize <br>
...
</center>

If we reach the end of the table we rollover to the beginning. Let's look at an example of linear probing by inserting (89, 18, 49, 58, 69) into a hash table of size 10. The hash function we use will be h(x) = x mod 10. 
<img src="./files/Hashing/linear-probing.png" width="600"/>
As we can see after inserting 89, and 18 there are no collisions since the cells are empty. The first collision occurs when inserting 49. As we can see h(49) = 9 but index 9 already has element 89 in it. So we will increase the index by 1 until we find an empty index. As we can see 0 is the first empty index (since 9 is the end of the table we roll back to the beginning of the table, 0) and we insert 49 into index 0. Now we try to insert 58 but h(58) = 8 and index 8 is not empty. Again we continually increase the index by 1 until we find a an empty index. We have to increase it 3 times as it collides with 89 then 49 while probing until finding that index 1 is empty. Lastly we insert 69 and see collision because h(69) = 9. Similar to 58 we have to probe 3 different indexs because index 0 has 49 and index 1 has 58. We finally find index 2 is empty and insert 69 into index 2. The code to insert and search using linear probing is as follows:
```python
def insert(x):
    # get the index based on the hash function
    index = hash(x)
    # Search for the next unused index. If we reach the end of the table roll back to the beginning
    while(hash_table[index] != None):
        index = (index + 1) % len(hash_table)
    
    hash_table[index] = x
    
# search is similar to insert 
# except if we see an empty index that means the element does not exist in the table    
def search(x):
    # get the index based on the hash function
    index = hash(x)
    # search for the key and if the index is empty it does not exist
    while(hash_table[index] != x):
        if hash_table[index] == None:
            return None
        index = (index + 1) % len(hash_table)
    
    return hash_table[index]
```

As long as the table is not full a free index can always be found but if the table is almost full finding an empty index could be O(n) for having to probe the entire table for the last empty index. 

### Quadratic Probing
**Quadratic probing** is similar to linear probing except the interval at which you probe the table. Instead of checking a fixed interval it is now quadratic in the number of succcessive probes in the table. Let us look at how we would calculate the index for an entry if the index is already occupied. The probe sequence is as follows: <br>
<center>
index = h(x) % tableSize <br>
index = (h(x) + $1^2$) % tableSize <br>
index = (h(x) + $2^2$) % tableSize <br>
index = (h(x) + $3^2$) % tableSize <br>
...
</center>

and so on until an empty index is found. So lets look at the same example as last time but with quadratic probing instead.
<img src="./files/Hashing/quadratic-probing.png" width="600"/>
Similar to linear we don't see the first collision until adding 49. Hence we calculate the new index as index + $1^2$ mod 10. In this case index 0 is empty and we insert 49. Next 58 has a collision as index 8 is not empty. We update the index = index + $1^2$ % mod 10 but index 9 is also taken. We perform this again index = index + $2^2$ % 10 and see that index 2 is empty and insert 58. This took two successive probes similar to last time. However the difference is when inserting 69, in linear probing it took 3 successfive probes to find an empty index, instead in this case it only takes two, first probing index 0 then index 4 and inserting 69. This helps prevent the clustering of probes being close to each other that linear probing had. Lets look at example code for how insertion and searching is done in quadratic probing
```python
def insert(x):
    # get the index based on the hash function
    index = hash(x)
    # keep track of successive probes
    succ_probes = 1
    # Search for the next unused index. If we reach the end of the table roll back to the beginning
    while(hash_table[index] != None):
        index = (hash(x) + succ_probes ** 2) % len(hash_table)
        succ_probes += 1
        
    hash_table[index] = x
    
# search is similar to insert 
# except if we see an empty index that means the element does not exist in the table    
def search(x):
    # get the index based on the hash function
    index = hash(x)
    # keep track of successive probes
    succ_probes = 1
    # search for the key and if the index is empty it does not exist
    while(hash_table[index] != x):
        if hash_table[index] == None:
            return None
        index = (hash(x) + succ_probes ** 2) % len(hash_table)
        succ_probes += 1
    
    return hash_table[index]
```
In linear probing it was bad to let the table get almost full but for quadratic the downfall is much worse. There is no guarantee of finding an empty cell once the table gets more than half full, or even before the table gets half full if the table size is not prime. This is because at most half of the table can be used as alternative locations to resolve collisions.

##### Theorem 
If the hash table size is b (a prime greater than 3), it can be proven that the first *b/2* alternative locations including the initial location h(k) are all distinct and unique. Suppose we have two locations $h(k) + x^2$ mod b and $h(k) + y^2$ mod b, where $0 \le x,y \le b/2$. If these two locations point to the same key space, but x ≠ y. Then the following would have to be true, <br><br>
\begin{align}
h(k)+x^{2}=h(k)+y^{2}{\pmod  {b}} \\
x^{2}=y^{2}{\pmod  {b}} \\
x^{2}-y^{2}=0{\pmod  {b}} \\ 
(x-y)(x+y)=0{\pmod  {b}} \\
\end{align}

As b (table size) is a prime greater than 3, either (x - y) or (x + y) has to be equal to zero. Since x and y are unique, (x - y) cannot be zero. Also, since 0 ≤ x, y ≤ (b / 2), (x + y) cannot be zero. Thus, by contradiction, it can be said that the first (b / 2) alternative locations after h(k) are unique. So an empty key space can always be found as long as at most (b / 2) locations are filled, i.e., the hash table is not more than half full

### Double Hashing
The last collision resolution method is called **double hashing**. In the case to calculate the interval for probing we use a second hash function. After each successful hash we multiply the result of the second hash function by the number of sucessful hashes. So a successful probing sequence would look as follows (here $h_2$ is the second hash function): 
<center>
    index = $h_1(x)$ + 0 * $h_2(x)$ mod tableSize <br>
    index = $h_1(x)$ + 1 * $h_2(x)$ mod tableSize <br>
    index = $h_1(x)$ + 2 * $h_2(x)$ mod tableSize <br>
    ...
</center>

The issue here is if there is a poor choice for the second hash function it drastically hurt the performance. A function such as $h_2(x) = R - (x\, mod\, R)$ where R is a prime smaller than the tableSize. Lets look at the previous example used for linear and quadratic hashing and $h_2(x) = 7 - (7\, mod\, 10)$
<img src="./files/Hashing/double-hashing.png" width="600"/>
Similar to the previous 49 is where the first collision occurs. So $h_2(49) = 7 - (49\, mod\, 7) = 7 - 0 = 7$, so we probe index 6 and insert 49 since it's empty. Next for 58, $h_2(58) = 7 - (58\, mod\, 7) = 7 - 2 = 5$, so we probe index 3 and insert 58 since it's empty. Lastly 69, $h_2(69) = 7 - (69\, mod\, 7) = 7 - 6 = 1$, so we probe 0 and insert 69 since it's empty. As we can see unlike linear and quadratic probing for this example double hashing as on average 1 probe required. However we can easily find a bad example, let's say we entered 60 into the hash table. $h_2(49) = 7 - (60\, mod\, 7) = 7 - 4 = 3$, we probe index 3, which already has 58, we probe again at $2*3 = 6$ which already has 49, we probe again at $3*3 = 9$ which has 89, lastly we probe $4*3 = 12\, mod\, 10 = 2$ and find an empty index. 

## Removal in Open Addressing
As you've noticed we have only covered search and inserting into the hash table. There is a problem with removing keys in open addressing schemes. The problem is that if we completely remove a key we could prematurely end our search for a key when searching through a table since we end the search at the first empty index we see. To address this we adopt the strategy of **lazy deletion**. In lazy deletion we do not ever remove the key until the index needs to be overriden with a new value. To acheive this each index would hold a boolean value of if the key is active. So when we delete a key in the hash table we would change the boolean value to False. This way during searching we will not end our searching prematurely. Let's look at inserting, searching, and removing for double hashing with this implemented:
```python
def insert(x):
    # get the index based on the hash function
    index = hash(x)
    # keep track of successive probes
    succ_probes = 1
    # Search for the next unused index. If we reach the end of the table roll back to the beginning
    # Here we add a check if the node is active, if not override it with the new key
    while(hash_table[index] != None and hash_table[index].active):
        index = (hash(x) + succ_probes * second_hash(x)) % len(hash_table)
        succ_probes += 1
            
    hash_table[index].element = x
    # We need to make sure the node is active
    hash_table[index].active = True
    
# search is similar to insert 
# except if we see an empty index that means the element does not exist in the table    
def search(x):
    # get the index based on the hash function
    index = hash(x)
    # keep track of successive probes
    succ_probes = 1
    # search for the key and if the index is empty it does not exist
    while(hash_table[index].element != x):
        if hash_table[index] == None:
            return None
        index = (hash(x) + succ_probes * second_hash(x)) % len(hash_table)
        succ_probes += 1
    
    # we can still find the key if the cell is not active so we need to check
    if hash_table[index].active:
        return hash_table[index]
    else:
        return None
    
# Make a node inactive during removal 
# Similar to search in that we have to find the node first
def remove(x):
    # get the index based on the hash function
    index = hash(x)
    # keep track of successive probes
    succ_probes = 1
    # search for the key and if the index is empty it does not exist
    while(hash_table[index].element != x):
        if hash_table[index] == None:
            return None
        index = (hash(x) + succ_probes * second_hash(x)) % len(hash_table)
        succ_probes += 1
    
    # we can still find the key if the cell is not active so we need to check
    # if the cell is active we make it unactive
    if hash_table[index].active:
        hash_table[index].active = False
```

## Rehashing
If the table gets too full the running time of operations will start to no longer be constant time and insertions might fail for quadratic probing. A solution is to build another table that is twice as big (similar to what arrayList do when they get full). In this solution we would update the hash function with the new tableSize and recompute the hash of each (non deleted) element (in this case we can delete non active cells since we are recomputing the hash values). This entire operation is called **rehashing**. The runnning time of performing this operation is O(N), sicne there are N elements to rehash and the table size is roughly $2N$, but this is actually not bad at all since this operation happens very infrequently. The question is when to rehash. There are several methods that are used to determine this: 
<center>
    When the table is half full <br>
    When an insertion fails <br>
    When a table reaches a certain load factor
</center>

**Load factor**$(\lambda)$ is the ratio of elements in the hash table to the size. In other words $\lambda = N/$tableSize. 

# Examples of Hash Tables
Hash tables are used quite frequently. They are the underlying data structure for Maps(Maps: Java, maps: c++, dict: python). Most compilers use them as **symbol tables** which keep track of which variable holds what value. 

Lets look at some sample problems we can solve use hashing *(in these examples we will be using pythons implementation of a hash table called a dictionary)*

1. Given an array of integers return indicies of the two numbers such that they add up to a specific target. You can assume that there is only one solution and you cannot use the same index twice.<br>
```
Example: Given nums = [2,7,11,15], target = 9.
Because nums[0] + nums[1] = 9 return [0,1]
``` 
<br><br>

2. Given a string, find the length of the longest substring without repeating characters.<br>
```
Example 1:
Input: "abcabcbb"
Output: 3
Explanation: The answer is "abc", with the length of 3.
-
Example 2:
Input: "bbbbb"
Output: 1
Explanation: The answer is "b", with the length of 1.
```

In [1]:
def two_sums(arr, target):
    table = dict()
    for i in range(len(arr)):
        compliment = target - arr[i]
        if compliment in table:
            return [i, table[compliment]]
        else:
            table[arr[i]] = i
    return []

def longest_substring(string):
    table = dict()
    max_sub = 0
    i, j = 0,0
    while j < len(string):
        if string[j] not in table:
            table[j] = 0
            j += 1
            max_sub = max(j-i+1, max_sub)
        elif string[j] in table:
            del table[string[i]]
            i += 1
            
    return max_sub